# Inicjalizacja kontekstu użytkownika

**Cel**: Konfiguracja środowiska i izolacja zasobów.

Skrypt realizuje strategię **Catalog Isolation** (rekomendowana):
1.  Tworzy dedykowany katalog dla użytkownika: `ecommerce_platform_<user_slug>` (np. `ecommerce_platform_kzb`).
2.  Wewnątrz katalogu tworzy standardowe schematy warstw: `bronze`, `silver`, `gold`.
3.  Ustawia zmienne globalne (`CATALOG`, `BRONZE_SCHEMA` itp.) używane w notebookach.

*Jeśli użytkownik nie ma uprawnień do tworzenia katalogów, skrypt automatycznie przełączy się na tryb izolacji schematami w katalogu `ecommerce_platform`.*

**Uruchom tę komórkę raz na początku sesji.**

## 📁 Architektura danych w szkoleniu

Szkolenie używa **dwóch podejść do przechowywania danych** w zależności od dnia:

### Dzień 1-2: Lokalne pliki → Delta Tables
- **Źródło**: Folder `dataset/` (CSV, JSON, Parquet)
- **Zmienna**: `DATASET_BASE_PATH` 
- **Cel**: Załadowanie do Delta tables w schematach Bronze/Silver/Gold
- **Przykład**: `spark.read.csv(f"{DATASET_BASE_PATH}/customers/customers.csv")`

### Dzień 3: Unity Catalog Volumes
- **Źródło**: Unity Catalog Volumes (`/Volumes/ecommerce_platform_<user>/default/kion_datasets`)
- **Zmienna**: `VOLUMES_BASE_PATH` (opcjonalne dla Dzień 3)
- **Cel**: Demonstracja zaawansowanych feature'ów UC (DLT, Lakeflow, Governance)
- **Przykład**: `spark.read.csv("/Volumes/ecommerce_platform_kzb/default/kion_datasets/customers.csv")`

### Dlaczego dwa podejścia?
- **Progresja edukacyjna**: Od prostych plików (Dzień 1) → Delta Lake (Dzień 2) → Unity Catalog (Dzień 3)
- **Real-world scenarios**: W produkcji często używa się Volumes dla managed data access w UC
- **Best practices**: Dzień 3 pokazuje, jak zarządzać danymi w enterprise environment

---

In [0]:
# === Konfiguracja katalogów i schematów ===
# Pobierz aktualnego użytkownika (dla środowiska produkcyjnego)
# raw_user = spark.sql("SELECT current_user()").first()[0]
raw_user = "trainer"  # Dla środowiska szkoleniowego

import re
user_slug = re.sub(
    r'[^a-z0-9]', 
    '_', 
    raw_user.lower()
)

print(f"User slug: {user_slug}")

# --- KONFIGURACJA ARCHITEKTURY ---
# Wybór strategii izolacji: 'CATALOG' (rekomendowana) lub 'SCHEMA' (fallback)
ISOLATION_MODE = 'CATALOG' 

if ISOLATION_MODE == 'CATALOG':
    # Opcja 1: Izolacja Katalogami (Czysty kod: FROM bronze.orders)
    # Wymaga uprawnień CREATE CATALOG
    CATALOG = f"ecommerce_platform_{user_slug}"
    BRONZE_SCHEMA = "bronze"
    SILVER_SCHEMA = "silver"
    GOLD_SCHEMA   = "gold"
else:
    # Opcja 2: Izolacja Schematami (Wspólny katalog: FROM user_bronze.orders)
    # Łatwiejsze uprawnienia
    CATALOG = "ecommerce_platform"
    BRONZE_SCHEMA = f"{user_slug}_bronze"
    SILVER_SCHEMA = f"{user_slug}_silver"
    GOLD_SCHEMA   = f"{user_slug}_gold"

# === Ścieżka do datasetów ===
# Dla lokalnego środowiska - ścieżka do folderu dataset w tym repo
#import os
#DATASET_BASE_PATH = os.path.abspath("../dataset")

 #Dla Databricks środowiska - Unity Catalog Volume:
# Zakładamy, że dane źródłowe są we wspólnym katalogu 'training_sources' lub podobnym
# Tutaj dla uproszczenia używamy katalogu użytkownika, ale w realnym scenariuszu byłby to read-only shared volume
DATASET_BASE_PATH = f"/Volumes/{CATALOG}/default/kion_datasets"


print(f"Dataset base path: {DATASET_BASE_PATH}")

# === Tworzenie katalogów i schematów (tylko w Databricks) ===
try:
    if ISOLATION_MODE == 'CATALOG':
        print(f"Creating/Using Catalog: {CATALOG}...")
        spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG}")
    
    spark.sql(f'USE CATALOG {CATALOG}')
    
    for s in [BRONZE_SCHEMA, SILVER_SCHEMA, GOLD_SCHEMA]:
        spark.sql(f'CREATE SCHEMA IF NOT EXISTS {CATALOG}.{s}')
    
    # Opcjonalnie: Utwórz wolumen na dane jeśli nie istnieje (dla celów szkoleniowych)
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.default")
    # spark.sql(f"CREATE VOLUME IF NOT EXISTS {CATALOG}.default.kion_datasets") 
    
    spark.sql(f'USE SCHEMA {BRONZE_SCHEMA}')
    print("✓ Unity Catalog resources created successfully")
except Exception as e:
    print(f"⚠ Error configuring Unity Catalog: {e}")
    print("Ensure you have CREATE CATALOG or CREATE SCHEMA permissions.")

# === Wyświetl informacje o katalogu ===
print("\n=== Informacje o katalogu ===")
display(
    spark.sql(f"DESCRIBE CATALOG EXTENDED {CATALOG}")
)

# === Podsumowanie konfiguracji użytkownika ===")
display(
    spark.createDataFrame(
        [
            (
                raw_user, 
                f'{CATALOG}.{BRONZE_SCHEMA}', 
                f'{CATALOG}.{SILVER_SCHEMA}', 
                f'{CATALOG}.{GOLD_SCHEMA}',
                DATASET_BASE_PATH
            )
        ],
        ['user', 'bronze_schema', 'silver_schema', 'gold_schema', 'dataset_path']
    )
)

print("\n✓ Inicjalizacja zakończona pomyślnie!")
print(f"✓ Tryb izolacji: {ISOLATION_MODE}")
print(f"✓ Katalog roboczy: {CATALOG}")
print(f"✓ Schematy: {BRONZE_SCHEMA}, {SILVER_SCHEMA}, {GOLD_SCHEMA}")
